In [ ]:
#Importamos librerias de utilidad
import numpy as np 
import pylab as pl
import random as rd
import copy

class TrafficSimulation():
    """
        Simulación de tráfico de carril N basada en Rickert et al. (1996)
        y el modelo original de Nagel & Schreckenberg (1992).

    """

    def __init__(self, len_road = 100, density = 0.01, max_vel = 5, 
                 prob_slowing = 0.5, prob_lane_change = 0.8, lanes = 1):
        self.len_road = len_road
        self.density = density
        self.max_vel = max_vel
        self.prob_slowing = prob_slowing
        self.prob_lane_change = prob_lane_change
        self.lanes = lanes

        self.car_counter = 0
        self.flow_counter = 0
        self.time_counter = 0

        self.config = []
        self.next_config = []


    def initialize(self):
        """
        Establece el estado inicial en función de la densidad y la longitud e
        inicia el conteo global de autos.
        """
        for _ in range(self.lanes):
            self.config.append(pl.zeros([self.len_road], dtype=int))
            self.next_config.append(pl.zeros([self.len_road], dtype=int))

        for lane in self.config:
            lane.fill(-1)
        for lane in self.next_config:
            lane.fill(-1)

        for state in self.config:
            indexes = np.random.choice(range(0, self.len_road),
                      size = int(self.density * self.len_road),
                      replace = False)
            state[indexes] = self.max_vel

        self.car_counter = self.count_cars()


    def lane_change(self):
        """
        Comprueba los cambios de carril y mueve los carros entre carriles.
        Rules:
        IF  v > gap
        AND v_o + 1 are empty AND empty behind >= vmax
        AND IF rd.rand < probability of switching
        THEN switch.
        """
        for lane_index, lane in enumerate(self.config):
            for spot in range(self.len_road):
                if lane[spot] != -1:                        #if car
                    switch = False                          #if car will switch lane
                    vel = lane[spot]

                    gap = 0                                 #rule 1 - check ahead            
                    looked_ahead = 0

                    while looked_ahead <= (self.max_vel):
                        if lane[(spot + gap + 1) % self.len_road] < 0:
                            gap += 1
                        else:
                            break
                        looked_ahead += 1

                    if gap < vel:                           #rule 1 - if no space ahead, switch
                        switch = True

                                                            #rule 2 - define neighbors
                    right_neighbor = self.config[(lane_index + 1) % self.lanes]
                    left_neighbor = self.config[(lane_index - 1) % self.lanes]

                    if switch:                              #rule 2 - if planning on switching
                        switch_right = False                
                        switch_left = False
                                                            #rule 2 - check for free lanes
                        if self.neighbor_free(right_neighbor, spot, vel):
                            switch_right = True
                        if self.neighbor_free(left_neighbor, spot, vel):
                            switch_left = True

                                                            #rule 3 - if less than random switching 
                                                            #probability, switch lanes
                        if rd.random() < self.prob_lane_change:
                            self.switch_lanes(lane_index, spot, vel, switch_right, switch_left)
                        else:
                            self.next_config[lane_index][spot] = vel

                                                            #if we're not switching the car, then we
                    else:                                   #have to write it back into it's own lane
                        self.next_config[lane_index][spot] = vel

        # self.state_difference()                                                    
        self.config = copy.deepcopy(self.next_config)        #update config and resetnext_config state
        for lane in self.next_config:                        
            lane.fill(-1)


    def neighbor_free(self, neighbor, cur_spot, vel):
        """
        Toma en un carril, carril vecino, punto actual del coche
        y es la velocidad y comprueba si el carril vecino tiene
        espacio suficiente para el coche. Devuelve True si es así, de lo contrario False.

        """
        free = True

        gap = 0                    
        check_back = -1 * self.max_vel      #distance to check in the back and front
        check_front = vel

        for index in range(check_back, check_front + 1):
            if neighbor[(cur_spot + index) % self.len_road] < 0:
                pass
            else:
                free = False
                break

        return free


    def switch_lanes(self, lane_index, spot, vel, right, left):
        """
        Determina si los coches se cambiarán a la derecha o
        carril izquierdo y actualiza el estado next_config.

        Debido a que estoy actualizando al estado next_config, de alguna manera
        veces los autos quieren cambiar al mismo lugar desde dos lados.
        
        Aquí, los autos de actualización anterior (carril izquierdo) tienen prioridad sobre
        los del carril derecho.

        """
                                                    #checking free slots in the config
                                                    #and the next_config lanes.
        if self.next_config[(lane_index + 1) % self.lanes][spot] == -1:
            right_taken = False
        else: right_taken = True    

        if self.next_config[(lane_index - 1) % self.lanes][spot] == -1:
            left_taken = False
        else: left_taken = True
                                                    #if both lanes free, pick randomly
        if (right and left) and not right_taken and not left_taken:                        
            if rd.random() < 0.5:
                self.next_config[(lane_index + 1) % self.lanes][spot] = vel
            else:
                self.next_config[(lane_index - 1) % self.lanes][spot] = vel
                                                            
        elif right and not right_taken:             #else, pick right or left as free
            self.next_config[(lane_index + 1) % self.lanes][spot] = vel
        elif left and not left_taken:
            self.next_config[(lane_index - 1) % self.lanes][spot] = vel

        else:                                        #if neither, keep car in lane
            self.next_config[lane_index][spot] = vel 


    def update_speed(self):
        """
        Itera sobre el array  y actualiza las velocidades de todos los autos.
        Rules: 
        IF v != vmax      THEN  v+=1
        IF v > gap        THEN  v=gap
        IF V>0 AND rand<p THEN  v-=1
        """
        for lane in self.config:
            for spot in range(self.len_road):
                if lane[spot] != -1:                  #if car
                    vel = lane[spot]

                    if vel < self.max_vel:            #rule 1
                        vel += 1

                    gap = 0                           #rule 2 - count gap size
                    looked_ahead = 0

                    while looked_ahead <= (self.max_vel):
                        if lane[(spot + gap + 1) % self.len_road] < 0:
                            gap += 1
                        else:
                            break
                        looked_ahead += 1


                    if vel > gap:                     #rule 2 - reduce speed to gap
                        vel = max(gap, 0)

                                                      #rule 3 - randomness
                    vel = max(vel - 1, 0) if rd.random() < self.prob_slowing else vel 

                    lane[spot] = vel                  #update speeds


def move_cars(self):
        """
        Mueve los autos hacia adelante y actualiza los estados de configuración.
        Crea una copia profunda del estado next_config y 
        luego lo borra después de mover todos los autos.

        """
        for lane, next_lane in zip(self.config, self.next_config):
            for spot in range(0, self.len_road):
                if lane[spot] != -1:                 #if car
                    distance = lane[spot]

                    next_lane[int(spot + distance) % self.len_road] = lane[spot]

        self.config = copy.deepcopy(self.next_config)
        for lane in self.next_config:
            lane.fill(-1)


    def count_flow(self):
        """
        Cuenta el número de coches que saldrán de la simulación.
        en el siguiente paso y actualiza el contador de tiempo.

        """
        for lane in self.config:
            for k in range(-1 * self.max_vel, 0):   #check last max_vel slots
                if lane[k] >= abs(k):               #count if car will move outside lane
                    self.flow_counter += 1

            self.time_counter += 1                  #add one time step


    def count_cars(self):
        """
        Cuenta la cantidad de autos en la simulación 
        para asegurarse de que no pierda ninguno en una dimensión 
        separada en el camino...
        
        """
        car_num = 0
        for lane in self.config:
            car_num += sum([1 if spot != -1 else 0 for spot in lane]) 

        return car_num

    def display(self, message = None):
        """
        Visualiza los estados de self.config como una cadena impresa.

        """
        if message:
            print(message)              #for debugging

        for lane in self.config:
            string = ''.join(['.' if x == -1 else str(x) for x in lane])
            print(string)

    def state_difference(self):
        """
        Función para ayudar al debud
        (muestra dónde se han producido cambios) entre estados. 
        Usar antes de actualizar la siguiente configuración.

        """
        state_dif = np.isclose(self.config, self.next_config)
        for lane in state_dif:
            string = ''.join(['.' if x == True else "X" for x in lane])
            print (string)

